In [ ]:
from datasets import load_from_disk

ds = load_from_disk("./data/russian_dialogues_20k")

In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import BitsAndBytesConfig
from datasets import load_from_disk
import torch
import os
import torch

In [ ]:
def load_model_and_tokenizer(model_name: str):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left"

    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16,
    )

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16,
        quantization_config=bnb_config,
    )

    model.config.use_cache = False
    model.config.pretraining_tp = 1

    return tokenizer, model

In [ ]:
def apply_lora(model):
    model = prepare_model_for_kbit_training(model)

    lora_config = LoraConfig(
        r=8,
        lora_alpha=16,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM"
    )

    model = get_peft_model(model, lora_config)
    model.gradient_checkpointing_enable()  

    return model

In [ ]:
def tokenize_russian_dialogues(example, tokenizer, max_length=512):
    prompt = example["prompt"].strip()
    response = example["response"].strip()

    text = (
        "<|system|>\nТы полезный AI-ассистент.\n"
        f"<|user|>\n{prompt}\n"
        f"<|assistant|>\n{response}"
    )

    return tokenizer(
        text,
        truncation=True,
        padding="max_length",
        max_length=max_length
    )


In [ ]:
def tokenize_dataset(dataset, tokenizer, tokenize_fn, max_length=512):
    def wrapped(example):
        return tokenize_fn(example, tokenizer, max_length=max_length)

    return dataset.map(wrapped, batched=True, remove_columns=dataset.column_names)

In [ ]:
def train_lora(
    model_name: str,
    dataset_path: str,
    output_dir: str,
    epochs: int = 3,
    batch_size: int = 4,
    lr: float = 2e-4,
    max_length: int = 512,
    tokenize_fn=None
):
    # Загрузка
    tokenizer, model = load_model_and_tokenizer(model_name)
    model = apply_lora(model)
    dataset = load_from_disk(dataset_path)
    dataset = tokenize_dataset(
        dataset["train"],
        tokenizer=tokenizer,
        tokenize_fn=tokenize_fn,
        max_length=max_length
    )

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False  # обязательно для CausalLM
    )


    torch.backends.cudnn.allow_tf32 = True
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cuda.matmul.allow_fp16_reduced_precision_reduction = True
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.deterministic = False


    # Аргументы обучения
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=4,
        lr_scheduler_type="cosine",
        warmup_steps=100,
        num_train_epochs=epochs,
        learning_rate=lr,
        fp16=True,
        logging_steps=20,
        save_steps=200,
        save_total_limit=2,
        report_to="none",
        evaluation_strategy="no"
    )

    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        tokenizer=tokenizer,
        data_collator=data_collator  
    )

    # Обучение
    trainer.train()
    model.save_pretrained(os.path.join(output_dir, "lora"))

In [ ]:
train_lora(
    model_name="TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    dataset_path="./data/russian_dialogues_20k",
    output_dir="./checkpoints/step_qa",
    epochs=3,
    batch_size=4,
    tokenize_fn=tokenize_russian_dialogues
)